# Czyszczenie danych i przygotowanie zbioru do analizy
Zadaniem do wykonania w tym projekcie jest oczyszczenie, a także usunięcie problemów w dostarczonym zbiorze TitatnicMess.tsv. 
Dane należy uporządkować, a także opisać napotkane problemy

Pierwszym krokiem w pracy przy zbiorze danych jest import potrzebnych bibliotek, które będą wykorzystywane podczas czyszczenia zbioru.

In [334]:
import locale
import numpy as np
import pandas as pd


Kolejnym krokiem jest wczytanie podanego zbioru danych TitanicMess.tsv z odpowiednim separatorem oraz kolumną indeksującą.

In [335]:
titanic = pd.read_csv('TitanicMess.tsv', sep='\t', index_col='PassengerId') 
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


Po wstępnej analizie wczytanych danych można dojść do wniosku, iż kolumna **Ship** posiada tylko jedną wartość, którą jest 'Titanic'. Wydaje się, że ta kolumna nie będzie potrzebna w dalszej analizie, ponieważ analizujemy dane tylko i wyłącznie ze statku Titanic. Najrozsądniejszym wyborem będzie usunięcie tej kolumny ze zbioru.

In [336]:
clear = titanic.drop(columns='ship')
clear.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S


W kolumnie **Sex** występuje sporo błędów, które potocznie można określić jako literówki. Aby uporządkować dane w tej kolumnie należy najpierw odczytać wszystkie unikalne wartości, które znajdują się tej tabeli. 

In [337]:
types = clear['Sex'].unique() 
print(types)

['male' 'female' 'malef' 'mal' 'fem' 'femmale']


Zauważyć można, iż rzeczywiście występują tak zwane literówki. Można łatwo dostrzec, które z podanych wartości powinny mieć wartość 'male', a które 'female' i zmienić je na te poprawne.

In [338]:
clear['Sex'] = clear['Sex'].replace(['femmale'],'female') 
clear['Sex'] = clear['Sex'].replace(['fem'],'female') 
clear['Sex'] = clear['Sex'].replace(['mal'],'male') 
clear['Sex'] = clear['Sex'].replace(['malef'],'male') 
types = clear['Sex'].unique() 
print(types)

['male' 'female']


Kolejnym krokiem jest sprawdzenie czy kolumny w zbiorze danych posiadają puste wartości.

In [339]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

Zastosowana funkcja zwróciła nam informację o zsumowanej ilości pustych (NULL) danych. Danych brakuje w trzech kolumnach: **Age**, **Cabin** oraz **Embarked**, w których brakuje odpowiednio 173, 685, 2 wartości.

Kolejnym krokiem jest sprawdzenie typów danych kolumn w zbiorze danych.

In [340]:
titanic.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

Z wyniku działania funkcji można zwrócić uwagę na typ tabeli **Age**, która posiadaja typ danych **object**. Natomiast jest to kolumna, która powinny zawierać tylko i wyłącznie liczby, co może świadczyć o tym, że w tej kolumnie występują dane nieprawidłowe.

In [341]:

for c, row in clear.iterrows():
    try:
        clear.at[c, 'Age'] = float(row['Age'])       
    except ValueError:
        clear.at[c, 'Age'] = np.NaN

clear[['Age']] = clear[['Age']].astype(float).round(2)
clear.dtypes


Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare         object
Cabin        object
Embarked     object
dtype: object

Kolejnym krokiem jest oczyszczenie kolumny **Age**, która jak było analizowane wcześniej, posiada puste dane. Dane trzeba uzupełnić. Najlepszą metodą będzie uzupełnienie danych średnią wartością kolumny.

In [342]:
clear['Age'] = clear['Age'].fillna(round(clear['Age'].mean()))
for index, row in clear.iterrows():
    print(row["Age"])


22.0
38.0
26.0
35.0
35.0
36.0
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
36.0
31.0
36.0
35.0
34.0
15.0
4.0
8.0
38.0
36.0
19.0
36.0
36.0
40.0
0.9
36.0
66.0
28.0
42.0
36.0
21.0
18.0
14.0
40.0
27.0
36.0
3.0
19.0
36.0
36.0
36.0
36.0
18.0
7.0
21.0
49.0
29.0
65.0
36.0
21.0
36.0
5.0
11.0
22.0
38.0
45.0
4.0
36.0
36.0
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
36.0
36.0
36.0
30.0
22.0
29.0
36.0
28.0
17.0
33.0
16.0
36.0
23.0
24.0
29.0
20.0
46.0
26.0
59.0
36.0
71.0
23.0
34.0
34.0
28.0
36.0
21.0
33.0
37.0
28.0
21.0
36.0
38.0
36.0
47.0
36.0
22.0
20.0
17.0
21.0
36.0
29.0
24.0
2.0
21.0
36.0
36.0
36.0
54.0
12.0
36.0
24.0
36.0
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
36.0
22.0
24.0
19.0
18.0
19.0
27.0
9.0
36.0
42.0
51.0
22.0
36.0
36.0
36.0
51.0
16.0
30.0
36.0
36.0
44.0
40.0
26.0
17.0
1.0
9.0
36.0
45.0
36.0
28.0
61.0
4.0
1.0
21.0
56.0
18.0
36.0
50.0
30.0
36.0
36.0
36.0
9.0
1.0
4.0
36.0
36.0
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
36.0
42.0
36.0
24.0
28.0
36.0
34.0
36.0
1

Kontynuując analizę zbioru można dostrzec również, że tak jak kolumna **Ship**, tak samo kolumna **Cabin** nie posiada żadnych użytecznych danych, które mogłyby później zostać użyte w analizie, dlatego kolumna ta zostanie usunięta.

In [343]:
clear.drop('Cabin', axis = 1, inplace = True) 
clear.head() 


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",S


Kolumna **Embarked** posiada 2 puste rekordy jak można było zobaczyć na początku analizy. Należy je uzupełnić. W tym celu została znaleziona najczęściej występująca wartość, która została wpisana w brakujących rekordach

In [344]:
clear['Embarked'].value_counts().idxmax() 


'S'

Kolumna **Embarked** została uzupełniona wartością 'S'.

In [345]:
fill = clear['Embarked'].value_counts().idxmax() 
clear['Embarked'] = clear['Embarked'].fillna('fill') 


Ostatnim krokiem po oczyszczeniu zbioru danych jest zapisanie oczyszczonego zbioru jako plik o nazwie **TitanicCleaned** o rozszerzeniu **.tsv**.

In [346]:
clear.to_csv('TitanicCleaned.tsv', sep = '\t')

W celu weryfikacji czy wszystkie dane zostały uzupełnione, wczytany zostanie wcześniej zapisany plik z oczyszczonym zbiorem danych. Zostanie również wywołana funkcja odczytująca czy znajdują się jakiekolwiek puste dane w całym zbiorze.

In [347]:
titanic2 = pd.read_csv('TitanicCleaned.tsv', sep='\t', index_col='PassengerId') 
titanic2.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",S


In [348]:
titanic2.isnull().sum() 


Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

Jak można zauważyć, wszystkie wartości w zbiorze danych zostały uzupełnione.